## Домашнее задание по Автоматической обработке естественного языка №2

### Настя Шабаева БКЛ181

#### Сравнение нескольких морфологических анализаторов

## Часть 1. Русский язык

### Импорт всего необходимого

In [61]:
import nltk
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymystem3 import Mystem
m = Mystem()
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

from sklearn.metrics import accuracy_score
import pandas as pd
pd.options.display.max_rows = 150
pd.set_option('display.expand_frame_repr', False)

### Сам текст

Я постаралась добавить в текст несколько проблемных моментов, с которыми у морфологических анализаторов могут возникнуть трудности при определении части речи. В основном, это случаи частеречной омонимии, например, 'вечером' - существительное и 'вечером' - наречие, 'пропасть' - существительное и  'пропасть' - инфинитив, 'больной' как существительное и некоторые другие. Также я добавила несколько слэнговых слов, таких как: чика, чувак, краш, рофлить, кринж, зашквар, хейтер. Возможно, с ними у анализаторов тоже возникнут некоторые трудности.
По-моему, ещё возможны некоторые трудности с местоимениями в разных контекстах. Например, "её" в разных контекстах может быть заменой существительного, а в некоторых - прилагательного (некоторые анализаторы размечают их по-разному).

In [62]:
rus_text = 'Давай вместе будем любоваться этим вечером. Чтобы не опоздать мне пришлось выехать вечером. Сегодня мама мыла фрукты без мыла. За деревней в лесу есть криповая пропасть, образованная много лет назад. Старожилы говорят, что даже самая образованная и подготовленная путешественница может там пропасть. Вася нарвал цветов на лугу. Из воды вынырнул нарвал. Больному за завтраком не дали конфет. Чувак стоял и любовался на заоблачные дали. В школе у меня был одноклассник бурят. Все люди, которые сейчас бурят скважину на том конце участка, - моя родня. Болотного цвета волосы чики развивались на ветру, родня её с Кикиморой. Моя килограмм слив, он свистел себе под нос. Утром небо было безоблачно и ясно. По этому вопросу мне уже всё ясно. Ему казалось, что он рофлит, а все подумали, что это полный зашквар и кринж. Её краш орал со своих хейтеров.'
print(rus_text)

Давай вместе будем любоваться этим вечером. Чтобы не опоздать мне пришлось выехать вечером. Сегодня мама мыла фрукты без мыла. За деревней в лесу есть криповая пропасть, образованная много лет назад. Старожилы говорят, что даже самая образованная и подготовленная путешественница может там пропасть. Вася нарвал цветов на лугу. Из воды вынырнул нарвал. Больному за завтраком не дали конфет. Чувак стоял и любовался на заоблачные дали. В школе у меня был одноклассник бурят. Все люди, которые сейчас бурят скважину на том конце участка, - моя родня. Болотного цвета волосы чики развивались на ветру, родня её с Кикиморой. Моя килограмм слив, он свистел себе под нос. Утром небо было безоблачно и ясно. По этому вопросу мне уже всё ясно. Ему казалось, что он рофлит, а все подумали, что это полный зашквар и кринж. Её краш орал со своих хейтеров.


### Словарь с моей разметкой

Ключами я сделала просто порядковые номера для того, чтобы слова с одинаковым написанием не сливались в один ключ. Например, в моем тексте есть слово "нарвал", которое используется как глагол, а также есть слово "нарвал" - существительное (животное). Чтобы они отображались в словаре по-разному, ключами я решила сделать порядковые номера.

Кроме того, поскольку в разных анализаторах ипользуются разные частеречные тегги, в своём словаре я разметила слова в терминах сразу всех анализаторов, чтобы сравнивать было удобнее.

In [86]:
rus_token_dict = {1 : {'давай' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  2 : {'вместе' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  3 : {'будем' : {'pymorphy':'VERB', 'natasha':'AUX', 'mystem':'V'}},
                  4 : {'любоваться' : {'pymorphy':'INFN', 'natasha':'VERB', 'mystem':'V'}},
                  5 : {'этим' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  6 : {'вечером' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  7 : {'чтобы' : {'pymorphy':'CONJ', 'natasha':'SCONJ', 'mystem':'CONJ'}},
                  8 : {'не' : {'pymorphy':'PRCL', 'natasha':'PART', 'mystem':'PART'}},
                  9 : {'опоздать' : {'pymorphy':'INFN', 'natasha':'VERB', 'mystem':'V'}},
                  10 : {'мне' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  11 : {'пришлось' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  12 : {'выехать' : {'pymorphy':'INFN', 'natasha':'VERB', 'mystem':'V'}},
                  13 : {'вечером' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  14 : {'сегодня' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  15 : {'мама' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  16 : {'мыла' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  17 : {'фрукты' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  18 : {'без' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  19 : {'мыла' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  20 : {'за' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  21 : {'деревней' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  22 : {'в' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  23 : {'лесу' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  24 : {'есть' : {'pymorphy':'INFN', 'natasha':'VERB', 'mystem':'V'}},
                  25 : {'криповая' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'A'}},
                  26 : {'пропасть' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  27 : {'образованная' : {'pymorphy':'PRTF', 'natasha':'VERB', 'mystem':'V'}},
                  28 : {'много' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  29 : {'лет' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  30 : {'назад' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  31 : {'старожилы' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  32 : {'говорят' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  33 : {'что' : {'pymorphy':'CONJ', 'natasha':'SCONJ', 'mystem':'CONJ'}},
                  34 : {'даже' : {'pymorphy':'PRCL', 'natasha':'PART', 'mystem':'PART'}},
                  35 : {'самая' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  36 : {'образованная' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'A'}},
                  37 : {'и' : {'pymorphy':'CONJ', 'natasha':'CCONJ', 'mystem':'CONJ'}},
                  38 : {'подготовленная' : {'pymorphy':'PRTF', 'natasha':'VERB', 'mystem':'V'}},
                  39 : {'путешественница' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  40 : {'может' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  41 : {'там' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADVPRO'}},
                  42 : {'пропасть' : {'pymorphy':'INFN', 'natasha':'VERB', 'mystem':'V'}},
                  43 : {'Вася' : {'pymorphy':'NOUN', 'natasha':'PROPN', 'mystem':'S'}},
                  44 : {'нарвал' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  45 : {'цветов' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  46 : {'на' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  47 : {'лугу' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  48 : {'из' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  49 : {'воды' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  50 : {'вынырнул' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  51 : {'нарвал' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  52 : {'больному' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  53 : {'за' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  54 : {'завтраком' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  55 : {'не' : {'pymorphy':'PRCL', 'natasha':'PART', 'mystem':'PART'}},
                  56 : {'дали' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  57 : {'конфет' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  58 : {'чувак' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  59 : {'стоял' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  60 : {'и' : {'pymorphy':'CONJ', 'natasha':'CCONJ', 'mystem':'CONJ'}},
                  61 : {'любовался' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  62 : {'на' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  63 : {'заоблачные' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'A'}},
                  64 : {'дали' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  65 : {'в' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  66 : {'школе' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  67 : {'у' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  68 : {'меня' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  69 : {'был' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  70 : {'одноклассник' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  71 : {'бурят' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  72 : {'все' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  73 : {'люди' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  74 : {'которые' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  75 : {'сейчас' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  76 : {'бурят' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  77 : {'скважину' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  78 : {'на' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  79 : {'том' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  80 : {'конце' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  81 : {'участка' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  82 : {'моя' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  83 : {'родня' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  84 : {'болотного' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'A'}},
                  85 : {'цвета' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  86 : {'волосы' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  87 : {'чики' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  88 : {'развевались' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  89 : {'на' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  90 : {'ветру' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  91 : {'родня' : {'pymorphy':'GRND', 'natasha':'VERB', 'mystem':'V'}},
                  92 : {'её' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  93 : {'с' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  94 : {'кикиморой' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  95 : {'моя' : {'pymorphy':'GRND', 'natasha':'VERB', 'mystem':'V'}},
                  96 : {'килограмм' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  97 : {'слив' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  98 : {'он' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  99 : {'свистел' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  100 : {'себе' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  101 : {'под' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  102 : {'нос' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  103 : {'утром' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  104 : {'небо' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  105 : {'было' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  106 : {'безоблачно' : {'pymorphy':'ADJS', 'natasha':'ADJ', 'mystem':'A'}},
                  107 : {'и' : {'pymorphy':'CONJ', 'natasha':'CCONJ', 'mystem':'CONJ'}},
                  108 : {'ясно' : {'pymorphy':'ADJS', 'natasha':'ADJ', 'mystem':'A'}},
                  109 : {'по' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  110 : {'этому' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  111 : {'вопросу' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  112 : {'мне' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  113 : {'уже' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  114 : {'всё' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  115 : {'ясно' : {'pymorphy':'ADVB', 'natasha':'ADV', 'mystem':'ADV'}},
                  116 : {'ему' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  117 : {'казалось' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  118 : {'что' : {'pymorphy':'CONJ', 'natasha':'SCONJ', 'mystem':'CONJ'}},
                  119 : {'он' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  120 : {'рофлит' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  121 : {'а' : {'pymorphy':'CONJ', 'natasha':'CCONJ', 'mystem':'CONJ'}},
                  122 : {'все' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  123 : {'подумали' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  124 : {'что' : {'pymorphy':'CONJ', 'natasha':'SCONJ', 'mystem':'CONJ'}},
                  125 : {'это' : {'pymorphy':'NPRO', 'natasha':'PRON', 'mystem':'SPRO'}},
                  126 : {'полный' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'A'}},
                  127 : {'зашквар' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  128 : {'и' : {'pymorphy':'CONJ', 'natasha':'CCONJ', 'mystem':'CONJ'}},
                  129 : {'кринж' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  130 : {'её' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  131 : {'краш' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}},
                  132 : {'орал' : {'pymorphy':'VERB', 'natasha':'VERB', 'mystem':'V'}},
                  133 : {'со' : {'pymorphy':'PREP', 'natasha':'ADP', 'mystem':'PR'}},
                  134 : {'своих' : {'pymorphy':'ADJF', 'natasha':'ADJ', 'mystem':'APRO'}},
                  135 : {'хейтеров' : {'pymorphy':'NOUN', 'natasha':'NOUN', 'mystem':'S'}}}

### Определение Accuracy:

Создаются 2 пустых списка: для теггов морфологического анализатора (pymorphy/mystem/Natasha) и для моих теггов. Проходим по списку токенов текста, определяем часть речи буквенных токенов, добавляем тэги в список.

Проходим по моему словарю и добавляем во второй список все тегги с ключом соответственно 'pymorphy', 'mystem' или 'natasha'.

На основе полученных списков считаем Accuracy.

In [64]:
pymorphy_list = []
real_list_pymorphy = []
for i in nltk.word_tokenize(rus_text):
    if i.isalpha():
        pymorphy_list.append(str(morph.parse(i.lower())[0].tag.POS))

for j in rus_token_dict.values():
    for i in j.values():
        real_list_pymorphy.append(i['pymorphy'])

print('Accuracy: ', accuracy_score(real_list_pymorphy, pymorphy_list))

Accuracy:  0.8296296296296296


In [59]:
mystem_list = []
real_list_mystem = []

for ana in m.analyze(rus_text):
    if 'analysis' in ana:
        gr = ana['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_list.append(pos)

for j in rus_token_dict.values():
    for i in j.values():
        real_list_mystem.append(i['mystem'])

print('Accuracy: ', accuracy_score(real_list_mystem, mystem_list))

Accuracy:  0.9037037037037037


In [87]:
natasha_list = []
real_list_natasha = []

doc = Doc(rus_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for i in doc.tokens:
    if i.pos != 'PUNCT':
        natasha_list.append(i.pos)

for j in rus_token_dict.values():
    for i in j.values():
        real_list_natasha.append(i['natasha'])

print('Accuracy: ', accuracy_score(real_list_natasha, natasha_list))

Accuracy:  0.8296296296296296


Возможно, с Natasha возникла некоторая неточность, поскольку я не до конца поняла, в каких контекстах проставляется тегг DET.




### Итог

Создаем таблицу, в которой отображаются все токены и все частеречные тегги (разных морфологических анализаторов и мои в терминах разных морфологических анализаторов). 

Выводим Accuracy всех анализаторов.

In [92]:
data = {'Word' : [], 'my POS Natasha' : [], 'Natasha POS' : [], 'my POS mystem' : [], 'mystem POS' : [], 'my POS pymorphy' : [], 'pymorphy POS' : []}

for i in range(len(natasha_list)):
    data['Word'].append(list(rus_token_dict[i+1].keys())[0])
    data['my POS Natasha'].append(real_list_natasha[i])
    data['Natasha POS'].append(natasha_list[i])
    data["my POS mystem"].append(real_list_mystem[i])
    data['mystem POS'].append(mystem_list[i])
    data["my POS pymorphy"].append(real_list_pymorphy[i])
    data['pymorphy POS'].append(pymorphy_list[i])

df = pd.DataFrame(data)

print(df, '\n')
print('Accuracy Natasha: ', accuracy_score(real_list_natasha, natasha_list))
print('Accuracy pymorphy: ', accuracy_score(real_list_pymorphy, pymorphy_list))
print('Accuracy mystem: ', accuracy_score(real_list_mystem, mystem_list))

                Word my POS Natasha Natasha POS my POS mystem mystem POS my POS pymorphy pymorphy POS
0              давай           VERB        VERB             V          V            VERB         VERB
1             вместе            ADV         ADV           ADV        ADV            ADVB         ADVB
2              будем            AUX         AUX             V          V            VERB         VERB
3         любоваться           VERB        VERB             V          V            INFN         INFN
4               этим            ADJ         DET          APRO       APRO            ADJF         NPRO
5            вечером           NOUN        NOUN             S          S            NOUN         ADVB
6              чтобы          SCONJ       SCONJ          CONJ       CONJ            CONJ         CONJ
7                 не           PART        PART          PART       PART            PRCL         PRCL
8           опоздать           VERB        VERB             V          V          

## Второй вариант

Изначально в своем словаре я приводила вариант тегга для каждого анализатора, но я не уверена, что это соответствует заданию. Поняла я это поздно, удалять первый вариант было жалко, поэтому я просто добавила второй вариант: в словаре каждому токену я присвоила удобный мне тегг, а затем приводила все тегги, полученные от морфологических анализаторов к этому виду.

### Новый словарь

In [93]:
rus_token_dict_new = {1 : {'давай' : 'VERB'},
                  2 : {'вместе' : 'ADV'},
                  3 : {'будем' : 'VERB'},
                  4 : {'любоваться' : 'VERB'},
                  5 : {'этим' : 'ADJ'},
                  6 : {'вечером' : 'NOUN'},
                  7 : {'чтобы' : 'CONJ'},
                  8 : {'не' : 'PART'},
                  9 : {'опоздать' : 'VERB'},
                  10 : {'мне' : 'NPRO'},
                  11 : {'пришлось' : 'VERB'},
                  12 : {'выехать' : 'VERB'},
                  13 : {'вечером' : 'ADV'},
                  14 : {'сегодня' : 'ADV'},
                  15 : {'мама' : 'NOUN'},
                  16 : {'мыла' : 'VERB'},
                  17 : {'фрукты' : 'NOUN'},
                  18 : {'без' : 'PREP'},
                  19 : {'мыла' : 'NOUN'},
                  20 : {'за' : 'PREP'},
                  21 : {'деревней' : 'NOUN'},
                  22 : {'в' : 'PREP'},
                  23 : {'лесу' : 'NOUN'},
                  24 : {'есть' : 'VERB'},
                  25 : {'криповая' : 'ADJ'},
                  26 : {'пропасть' : 'NOUN'},
                  27 : {'образованная' : 'VERB'},
                  28 : {'много' : 'ADV'},
                  29 : {'лет' : 'NOUN'},
                  30 : {'назад' : 'ADV'},
                  31 : {'старожилы' : 'NOUN'},
                  32 : {'говорят' : 'VERB'},
                  33 : {'что' : 'CONJ'},
                  34 : {'даже' : 'PART'},
                  35 : {'самая' : 'ADJ'},
                  36 : {'образованная' : 'ADJ'},
                  37 : {'и' : 'CONJ'},
                  38 : {'подготовленная' : 'VERB'},
                  39 : {'путешественница' : 'NOUN'},
                  40 : {'может' : 'VERB'},
                  41 : {'там' : 'ADV'},
                  42 : {'пропасть' : 'VERB'},
                  43 : {'Вася' : 'NOUN'},
                  44 : {'нарвал' : 'VERB'},
                  45 : {'цветов' : 'NOUN'},
                  46 : {'на' : 'PREP'},
                  47 : {'лугу' : 'NOUN'},
                  48 : {'из' : 'PREP'},
                  49 : {'воды' : 'NOUN'},
                  50 : {'вынырнул' : 'VERB'},
                  51 : {'нарвал' : 'NOUN'},
                  52 : {'больному' : 'NOUN'},
                  53 : {'за' : 'PREP'},
                  54 : {'завтраком' : 'NOUN'},
                  55 : {'не' : 'PART'},
                  56 : {'дали' : 'VERB'},
                  57 : {'конфет' : 'NOUN'},
                  58 : {'чувак' : 'NOUN'},
                  59 : {'стоял' : 'VERB'},
                  60 : {'и' : 'CONJ'},
                  61 : {'любовался' : 'VERB'},
                  62 : {'на' : 'PREP'},
                  63 : {'заоблачные' : 'ADJ'},
                  64 : {'дали' : 'NOUN'},
                  65 : {'в' : 'PREP'},
                  66 : {'школе' : 'NOUN'},
                  67 : {'у' : 'PREP'},
                  68 : {'меня' : 'NPRO'},
                  69 : {'был' : 'VERB'},
                  70 : {'одноклассник' : 'NOUN'},
                  71 : {'бурят' : 'NOUN'},
                  72 : {'все' : 'ADJ'},
                  73 : {'люди' : 'NOUN'},
                  74 : {'которые' : 'ADJ'},
                  75 : {'сейчас' : 'ADV'},
                  76 : {'бурят' : 'VERB'},
                  77 : {'скважину' : 'NOUN'},
                  78 : {'на' : 'PREP'},
                  79 : {'том' : 'ADJ'},
                  80 : {'конце' : 'NOUN'},
                  81 : {'участка' : 'NOUN'},
                  82 : {'моя' : 'ADJ'},
                  83 : {'родня' : 'NOUN'},
                  84 : {'болотного' : 'ADJ'},
                  85 : {'цвета' : 'NOUN'},
                  86 : {'волосы' : 'NOUN'},
                  87 : {'чики' : 'NOUN'},
                  88 : {'развевались' : 'VERB'},
                  89 : {'на' : 'PREP'},
                  90 : {'ветру' : 'NOUN'},
                  91 : {'родня' : 'VERB'},
                  92 : {'её' : 'NPRO'},
                  93 : {'с' : 'PREP'},
                  94 : {'кикиморой' : 'NOUN'},
                  95 : {'моя' : 'VERB'},
                  96 : {'килограмм' : 'NOUN'},
                  97 : {'слив' : 'NOUN'},
                  98 : {'он' : 'NPRO'},
                  99 : {'свистел' : 'VERB'},
                  100 : {'себе' : 'NPRO'},
                  101 : {'под' : 'PREP'},
                  102 : {'нос' : 'NOUN'},
                  103 : {'утром' : 'ADV'},
                  104 : {'небо' : 'NOUN'},
                  105 : {'было' : 'VERB'},
                  106 : {'безоблачно' : 'ADJ'},
                  107 : {'и' : 'CONJ'},
                  108 : {'ясно' : 'ADJ'},
                  109 : {'по' : 'PREP'},
                  110 : {'этому' : 'ADJ'},
                  111 : {'вопросу' : 'NOUN'},
                  112 : {'мне' : 'NPRO'},
                  113 : {'уже' : 'ADV'},
                  114 : {'всё' : 'NPRO'},
                  115 : {'ясно' : 'ADV'},
                  116 : {'ему' : 'NPRO'},
                  117 : {'казалось' : 'VERB'},
                  118 : {'что' : 'CONJ'},
                  119 : {'он' : 'NPRO'},
                  120 : {'рофлит' : 'VERB'},
                  121 : {'а' : 'CONJ'},
                  122 : {'все' : 'NPRO'},
                  123 : {'подумали' : 'VERB'},
                  124 : {'что' : 'CONJ'},
                  125 : {'это' : 'NPRO'},
                  126 : {'полный' : 'ADJ'},
                  127 : {'зашквар' : 'NOUN'},
                  128 : {'и' : 'CONJ'},
                  129 : {'кринж' : 'NOUN'},
                  130 : {'её' : 'ADJ'},
                  131 : {'краш' : 'NOUN'},
                  132 : {'орал' : 'VERB'},
                  133 : {'со' : 'PREP'},
                  134 : {'своих' : 'ADJ'},
                  135 : {'хейтеров' : 'NOUN'}}

### Список частей речи из моей разметки

In [94]:
my_pos_list = []

for i in rus_token_dict_new.values():
    for j in i.values():
        my_pos_list.append(j)

### Pymorphy

Проходим по списку токенов текста. Если токен буквенный, вычленяем частеречный тегг, далее преобразуем его название, если это необходимо идобавляем тегг в список. На основе полученного списка и списка с теггами из моего разбора считаем Accuracy

In [95]:
pymorphy_pos_list = []
same_tags = ['NOUN', 'VERB', 'CONJ', 'NPRO', 'PREP']

for i in nltk.word_tokenize(rus_text):
    if i.isalpha():
        pos = str(morph.parse(i.lower())[0].tag.POS)
        if pos in same_tags:
            pymorphy_pos_list.append(pos)
        else:
            if (pos == 'INFN') or (pos == 'GRND') or (pos == 'PRTF'):
                pymorphy_pos_list.append('VERB')
            else:
                if (pos == 'ADJF') or (pos == 'ADJS'):
                    pymorphy_pos_list.append('ADJ')
                else:
                    if pos == 'ADVB':
                        pymorphy_pos_list.append('ADV')
                    else:
                        if pos == 'PRCL':
                            pymorphy_pos_list.append('PART')
                        else:
                            pymorphy_pos_list.append(pos)

print('Accuracy: ', accuracy_score(my_pos_list, pymorphy_pos_list))

Accuracy:  0.8296296296296296


### Mystem

Проходим по кажому анализу в тексте. Если в анализе есть частеречный тегг, при необходимости заменяем его название и добавляем в список mystem_pos_list. 

Далее на основе этого списка и списка с частями речи из моей разметки считаем Accuracy

In [96]:
mystem_pos_list = []
same_tags_mystem  = ['ADV', 'CONJ', 'PART']

for ana in m.analyze(rus_text):
    if 'analysis' in ana:
        gr = ana['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        if pos in same_tags_mystem:
            mystem_pos_list.append(pos)
        else:
            if pos == 'V':
                mystem_pos_list.append('VERB')
            else:
                if pos == 'A':
                    mystem_pos_list.append('ADJ')
                else:
                    if pos == 'S':
                        mystem_pos_list.append('NOUN')
                    else:
                        if pos == 'APRO':
                            mystem_pos_list.append('ADJ')
                        else:
                            if pos == 'ADVPRO':
                                mystem_pos_list.append('ADV')
                            else:
                                if pos == 'SPRO':
                                    mystem_pos_list.append('NPRO')
                                else:
                                    if pos == 'PR':
                                        mystem_pos_list.append('PREP')

print('Accuracy: ', accuracy_score(my_pos_list, mystem_pos_list))

Accuracy:  0.9037037037037037


### Natasha

Обрабатываем весь текст, затем проходим по токенам, которые уже имеют разбор. Частеречные тегги, при необходимости заменяя названия, добавляем в список natasha_pos_list, на основе которого (вместе со списком my_pos_list) считаем Accuracy

In [97]:
natasha_pos_list = []
same_tags_natasha = ['NOUN', 'VERB', 'ADJ', 'ADV', 'PART']

doc = Doc(rus_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for i in doc.tokens:
    if i.pos != 'PUNCT':
        if i.pos in same_tags_natasha:
            natasha_pos_list.append(i.pos)
        else:
            if i.pos == 'PRON':
                natasha_pos_list.append('NPRO')
            else:
                if i.pos == 'ADP':
                    natasha_pos_list.append('PREP')
                else:
                    if (i.pos == 'CCONJ') or (i.pos == 'SCONJ'):
                        natasha_pos_list.append('CONJ')
                    else:
                        if i.pos == 'AUX':
                            natasha_pos_list.append('VERB')
                        else:
                            if i.pos == 'PROPN':
                                natasha_pos_list.append('NOUN')
                            else:
                                if i.pos == 'DET':
                                    natasha_pos_list.append('ADJ')

print('Accuracy: ', accuracy_score(my_pos_list, natasha_pos_list))

Accuracy:  0.8888888888888888


### Итог

Строим таблицу со всеми данными и в конце выводим Accuracy каждого морфологического анализатора.

В целом, видимо, разницы между двумя вариантами никакой нет. Единственное различие можно заметить в Accuracy Natasha, но скорее всего это связано с теггом DET (во втором варианте все слова с таким теггом считались за правильные, а в первом - нет).

In [98]:
data2 = {'Word' : [], 'my POS' : [], 'pymorphy POS' : [], 'mystem POS' : [], 'Natasha POS' : []}

for i in range(len(my_pos_list)):
    data2['Word'].append(list(rus_token_dict_new[i+1].keys())[0])
    data2['my POS'].append(my_pos_list[i])
    data2['pymorphy POS'].append(pymorphy_pos_list[i])
    data2['mystem POS'].append(mystem_pos_list[i])
    data2['Natasha POS'].append(natasha_pos_list[i])

df2 = pd.DataFrame(data2)

print(df2, '\n')
print('Accuracy pymorphy: ', accuracy_score(my_pos_list, pymorphy_pos_list))
print('Accuracy mystem: ', accuracy_score(my_pos_list, mystem_pos_list))
print('Accuracy Natasha: ', accuracy_score(my_pos_list, natasha_pos_list))

                Word my POS pymorphy POS mystem POS Natasha POS
0              давай   VERB         VERB       VERB        VERB
1             вместе    ADV          ADV        ADV         ADV
2              будем   VERB         VERB       VERB        VERB
3         любоваться   VERB         VERB       VERB        VERB
4               этим    ADJ         NPRO        ADJ         ADJ
5            вечером   NOUN          ADV       NOUN        NOUN
6              чтобы   CONJ         CONJ       CONJ        CONJ
7                 не   PART         PART       PART        PART
8           опоздать   VERB         VERB       VERB        VERB
9                мне   NPRO         NPRO       NPRO        NPRO
10          пришлось   VERB         VERB       VERB        VERB
11           выехать   VERB         VERB       VERB        VERB
12           вечером    ADV          ADV       NOUN        NOUN
13           сегодня    ADV          ADV        ADV         ADV
14              мама   NOUN         NOUN